In [1]:
# config
config = {
    "data_path": "../",
    "model": {
        "encoder_name": "timm-resnest26d",
        "loss_smooth": 1.0,
        "optimizer_params": {"lr": 0.003, "weight_decay": 0.0},
        "scheduler": {
            "name": "CosineAnnealingLR",
            "params": {
                "CosineAnnealingLR": {"T_max": 50, "eta_min": 1e-06, "last_epoch": -1},
                "ReduceLROnPlateau": {
                    "factor": 0.316,
                    "mode": "min",
                    "patience": 3,
                    "verbose": True,
                },
            },
        },
        "seg_model": "Unet",
    },
    "output_dir": "models",
    "progress_bar_refresh_rate": 500,
    "seed": 42,
    "train_bs": 128,
    "trainer": {
        "enable_progress_bar": True,
        "max_epochs": 50,
        "min_epochs": 20,
        "accelerator": "mps",
        "devices": 1,
    },
    "valid_bs": 128,
    "workers": 0,
    "device": "mps",
    "folds": {
        "n_splits": 4,
        "random_state": 42,
        "train_folds": [0, 1, 2, 3]
    }
}

import pandas as pd

train = pd.read_csv('data/single_turbine_data/train_reduced_unskewed.csv')
test = pd.read_csv('data/single_turbine_data/test_reduced_unskewed.csv')

label = ['1_Gear oil temperature (°C)']

X_train = train.drop(label, axis=1)
y_train = train[label]
X_test = test.drop(label, axis=1)
y_test = test[label]

# convert to datetime
X_train['# Date and time'] = pd.to_datetime(X_train['# Date and time'])
X_test['# Date and time'] = pd.to_datetime(X_test['# Date and time'])

# Setting the index
X_train.set_index('# Date and time', inplace=True)
X_test.set_index('# Date and time', inplace=True)

original_cols = [
    # '1_Wind direction (°)',
    #    '1_Nacelle position (°)', 
       '1_Power (kW)',
       '1_Front bearing temperature (°C)', '1_Rear bearing temperature (°C)',
       '1_Stator temperature 1 (°C)', '1_Nacelle ambient temperature (°C)',
       '1_Nacelle temperature (°C)', '1_Transformer temperature (°C)',
       '1_Generator bearing rear temperature (°C)',
       '1_Generator bearing front temperature (°C)', '1_Temp. top box (°C)',
       '1_Hub temperature (°C)', '1_Ambient temperature (converter) (°C)',
       '1_Rotor bearing temp (°C)', '1_Transformer cell temperature (°C)', '1_Generator RPM (RPM)']
extras = ['month_sin', 'month_cos', 'hour_sin', 'hour_cos', 
'curtailed', 
'offline',
]

cols = original_cols
X_test = X_test[cols]
X_train = X_train[cols]

# scale the data
from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
import lightning.pytorch as pl
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

# Reshape data for sequence model (e.g., RNN, GRU, LSTM)
sequence_length = 10  # Define the sequence length

def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        sequences.append(seq)
    return torch.stack(sequences)

X_train_tensor = torch.Tensor(X_train)
X_test_tensor = torch.Tensor(X_test)
y_train_tensor = torch.Tensor(y_train)
y_test_tensor = torch.Tensor(y_test)

X_train_seq = create_sequences(X_train_tensor, sequence_length)
y_train_seq = create_sequences(y_train_tensor, sequence_length)
X_test_seq = create_sequences(X_test_tensor, sequence_length)
y_test_seq = create_sequences(y_test_tensor, sequence_length)

train_dataset_seq = TensorDataset(X_train_seq, y_train_seq)
train_loader_seq = DataLoader(train_dataset_seq, batch_size=32, shuffle=False)

test_dataset_seq = TensorDataset(X_test_seq, y_test_seq)
test_loader_seq = DataLoader(test_dataset_seq, batch_size=32, shuffle=False)


# Define Neural Network
class LightningModule(pl.LightningModule):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.config = config

        self.numerical_linear  = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.LayerNorm(64)
        )
        
        self.rnn = nn.GRU(64, 128,
                          num_layers=2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(
            nn.Linear(128 * 2, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, output_dim))
        
        self.loss = nn.MSELoss()

    def forward(self, x):
        numerical_embedding = self.numerical_linear(x)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output


    def configure_optimizers(self):
        optimizer = AdamW(self.parameters())

        if self.config['model']["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config['model']["scheduler"]["params"][self.config['model']["scheduler"]["name"]],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config['model']["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config['model']["scheduler"]["params"][self.config['model']["scheduler"]["name"]],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat.reshape(-1, y_hat.shape[-1]), y.reshape(-1, y.shape[-1]))
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=False, on_epoch=True, prog_bar=True)
        logs = {"train_loss": loss, "lr": lr}
        return {"loss": loss, "log": logs}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        return {"val_loss": loss}


early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)
progress_bar_callback = TQDMProgressBar(refresh_rate=config["progress_bar_refresh_rate"])

from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")

trainer = pl.Trainer(
        callbacks=[early_stop_callback, progress_bar_callback],
         logger=logger, **config["trainer"]
    )

# Train the model
model = LightningModule(X_train.shape[1], y_train.shape[1]).to(torch.device('mps'))
trainer.fit(model, train_loader_seq, test_loader_seq)

# Test the model
model.eval()
y_pred = model(X_test_tensor)
y_pred = y_pred.detach().numpy()
y_pred = scaler.inverse_transform(y_pred)

# Evaluate the model
from sklearn.metrics import mean_squared_error
from math import sqrt

# unscale the data
y_test = scaler.inverse_transform(y_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: %.3f' % rmse)

# Plot the results
import matplotlib.pyplot as plt

plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

import Path
# Save results
results_file = Path('results.csv')
results_df = pd.DataFrame(
    {
        'Model': 'Neural Network',
        'Training RMSE': trainer.logged_metrics['train_loss'].item(),
        'Validation RMSE': trainer.logged_metrics['val_loss'].item(),
        'Iterations': trainer.global_step,
        'Learning Rate': config['model']['optimizer_params']['lr'],
        'Depth': [X_train.shape[1], 64, 64, y_train.shape[1]],
        'Loss Function': 'MSE',
        'Features': ', '.join(cols),
    }
)

if results_file.exists():
    existing_df = pd.read_csv(results_file)
    results_df = pd.concat([existing_df, results_df])

results_df.to_csv(results_file, index=False)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name             | Type       | Params
------------------------------------------------
0 | numerical_linear | Sequential | 1.2 K 
1 | rnn              | GRU        | 445 K 
2 | linear_out       | Sequential | 33.3 K
3 | loss             | MSELoss    | 0     
------------------------------------------------
479 K     Trainable params
0         Non-trainable params
479 K     Total params
1.919     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 0.734


Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.733
/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [ ]:
trainer.logged_metrics

In [ ]:
# Access logged metrics
train_loss = trainer.logged_metrics['train_loss']
val_loss = trainer.logged_metrics['val_loss']

# Plot
plt.figure()
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Test the model
model.eval()
y_pred = model(X_test_tensor)
y_pred = y_pred.detach().numpy()
y_pred = scaler.inverse_transform(y_pred)

# Evaluate the model
from sklearn.metrics import mean_squared_error
from math import sqrt

# unscale the data
y_test = scaler.inverse_transform(y_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: %.3f' % rmse)

# Plot the results
import matplotlib.pyplot as plt

plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
import numpy as np

train = pd.read_csv('data/single_turbine_data/train_reduced_unskewed.csv')

label = ['1_Gear oil temperature (°C)']

y_train = train[label].values

# Assuming y_pred and y_true are your prediction and ground truth numpy arrays

# Compute statistics of ground truth and predictions
mean_true = np.mean(y_train)
std_true = np.std(y_train)

mean_pred = np.mean(y_pred)
std_pred = np.std(y_pred)

# Rescale predictions
y_pred_rescaled = ((y_pred - mean_pred) / std_pred) * std_true + mean_true

In [ ]:
rmse = sqrt(mean_squared_error(y_test, y_pred_rescaled))
print('Test RMSE: %.3f' % rmse)

# Plot the results
import matplotlib.pyplot as plt

plt.plot(y_test, label='Actual')
plt.plot(y_pred_rescaled, label='Predicted')
plt.legend()
plt.show()